In [1]:
import ScraperFC as sfc
import pandas as pd
import os

In [2]:
sofascore = sfc.Sofascore()

In [3]:
year = "2025"
liga = 'Liga 1 Peru'

In [4]:
comps = {
    # Copas internaciones de países (hombres)
    'World Cup': 16, 'Euros': 1, 'Gold Cup': 140, 'Copa America': 133,
    # Copas Continentales de clubes
    'Champions League': 7, 'Europa League': 679, 'Europa Conference League': 17015, 
    "Copa Libertadores": 384, 'Copa Sudamericana': 480,
    # Ligas europeas
    'EPL': 17, 'La Liga': 8, 'Bundesliga': 35, 'Serie A': 23, 'Ligue 1': 34, 'Turkish Super Lig': 52,
    # Ligas de AMERICA DEL SUR
    'Argentina Liga Profesional': 155, 'Argentina Copa de la Liga Profesional': 13475,
    'Liga 1 Peru': 406,  'Chile Primera Division': 11653, 'Venezuela Primera Division': 231,
    'Uruguay Primera Division': 278 , 'Ecuador LigaPro':240, 'Brasileirão Série A':325,
    # Ligas de USA
    'MLS': 242, 'USL Championship': 13363, 'USL1': 13362, 'USL2': 13546,
    # Ligas de Mexico
    'Mexico LigaMX Apertura':11621, 'LigaMX Clausura': 11620,
    # Ligas de Medio Oriente
    "Saudi Pro League": 955,
    # Copas internacionales de mujeres
    "Women's World Cup": 290
}

In [5]:
partidos = sofascore.get_match_dicts(year=year, league=liga)

Running


In [6]:
print(partidos)

[{'correctAiInsight': False, 'correctHalftimeAiInsight': False, 'tournament': {'name': 'Liga 1, Clausura', 'slug': 'liga-1-clausura', 'category': {'name': 'Peru', 'slug': 'peru', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 20, 'country': {'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Peru', 'slug': 'peru'}, 'flag': 'peru', 'alpha2': 'PE', 'fieldTranslations': {'nameTranslation': {'ar': 'بيرو'}, 'shortNameTranslation': {}}}, 'uniqueTournament': {'name': 'Liga 1', 'slug': 'liga-1', 'secondaryColorHex': '#f91018', 'category': {'name': 'Peru', 'slug': 'peru', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 20, 'country': {'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Peru', 'slug': 'peru'}, 'flag': 'peru', 'alpha2': 'PE', 'fieldTranslations': {'nameTranslation': {'ar': 'بيرو'}, 'shortNameTranslation': {}}}, 'userCount': 19687, 'hasPerformanceGraphFeature': False, 'id': 406, 'country': {}, 'hasEventPlayerStatistics': True, 'displayInverseHomeAwayTeams': False

In [7]:
rows = []

for partido in partidos:
    match_id = str(partido.get('id', ''))

    # URL sin request extra
    home_slug = partido.get('homeTeam', {}).get('slug', '')
    away_slug = partido.get('awayTeam', {}).get('slug', '')
    custom_id = partido.get('customId', '')
    match_url = f"https://www.sofascore.com/{home_slug}-{away_slug}/{custom_id}#id:{match_id}"

    # Campos rápidos (del listado)
    home_name   = partido.get('homeTeam', {}).get('name')
    home_id     = partido.get('homeTeam', {}).get('id')
    away_name   = partido.get('awayTeam', {}).get('name')
    away_id     = partido.get('awayTeam', {}).get('id')
    home_score  = partido.get('homeScore', {}).get('current')
    away_score  = partido.get('awayScore', {}).get('current')

    hc = partido.get('homeTeam', {}).get('teamColors', {}) or {}
    ac = partido.get('awayTeam', {}).get('teamColors', {}) or {}
    home_colors = f"Primary: {hc.get('primary','')}, Secondary: {hc.get('secondary','')}"
    away_colors = f"Primary: {ac.get('primary','')}, Secondary: {ac.get('secondary','')}"

    tournament   = partido.get('tournament', {}).get('name')
    round_number = partido.get('roundInfo', {}).get('round')
    season       = partido.get('season', {}).get('name')

    # ---- Detalle (1 request por partido) ----
    md = sofascore.get_match_dict(match_id)

    ht = md.get('homeTeam', {}) or {}
    at = md.get('awayTeam', {}) or {}
    venue_dict = (md.get('venue') or ht.get('venue') or at.get('venue') or {})  # fallback

    # Asignaciones seguras
    manager_home = (ht.get('manager') or {}).get('name')
    manager_away = (at.get('manager') or {}).get('name')

    hs = md.get('homeScore', {}) or {}
    as_ = md.get('awayScore', {}) or {}

    goles_1T_home = hs.get('period1')
    goles_2T_home = hs.get('period2')
    goles_1T_away = as_.get('period1')
    goles_2T_away = as_.get('period2')

    disp_home = hs.get('display')
    disp_away = as_.get('display')
    resultado_final = f"{disp_home} - {disp_away}" if disp_home is not None and disp_away is not None else None

    estadio = venue_dict.get('name')
    ciudad  = (venue_dict.get('city') or {}).get('name')
    fecha   = pd.to_datetime(md.get('startTimestamp'), unit='s', errors='coerce')
    arbitro = (md.get('referee') or {}).get('name')

    # Construcción de la fila (sin .update con getters anidados)
    row = {
        'match_id'          : match_id,
        'match_url'         : match_url,
        'home'              : home_name,
        'home_id'           : home_id,
        'home_score'        : home_score,
        'away'              : away_name,
        'away_id'           : away_id,
        'away_score'        : away_score,
        'home_team_colors'  : home_colors,
        'away_team_colors'  : away_colors,
        'tournament'        : tournament,
        'round_number'      : round_number,
        'season'            : season,

        # resumen detallado
        'manager_home'      : manager_home,
        'manager_away'      : manager_away,
        'goles_1T_home'     : goles_1T_home,
        'goles_2T_home'     : goles_2T_home,
        'goles_1T_away'     : goles_1T_away,
        'goles_2T_away'     : goles_2T_away,
        'resultado_final'   : resultado_final,
        'estadio'           : estadio,
        'ciudad'            : ciudad,
        'fecha'             : fecha,
        'arbitro'           : arbitro,
    }

    rows.append(row)

# Tabla final en Jupyter
df_matches = pd.DataFrame(rows).reset_index(drop=True)
df_matches

,match_id,match_url,home,home_id,home_score,away,away_id,away_score,home_team_colors,away_team_colors,...,manager_away,goles_1T_home,goles_2T_home,goles_1T_away,goles_2T_away,resultado_final,estadio,ciudad,fecha,arbitro
0,14841186,https://www.sofascore.com/club-sporting-crista...,Club Sporting Cristal,2302,0.0,Los Chankas CYC,252254,1.0,"Primary: #33ccff, Secondary: #ffffff","Primary: #ffffff, Secondary: #e20e0e",...,Walter Paolella,0.0,0.0,1.0,0.0,0 - 1,Estadio Alberto Gallardo,Lima,2025-10-26 16:00:00,Augusto Menendez
1,14841191,https://www.sofascore.com/universidad-tecnica-...,Universidad Técnica de Cajamarca,87854,3.0,Alianza Universidad,306660,2.0,"Primary: #ffffff, Secondary: #ffffff","Primary: #ff0000, Secondary: #0000c8",...,Roberto Mosquera,0.0,3.0,2.0,0.0,3 - 2,Estadio Heroes de San Ramon,Cajamarca,2025-10-26 18:15:00,Christian Santos
2,14843657,https://www.sofascore.com/asociacion-deportiva...,Asociación Deportiva Tarma,335557,1.0,Universitario de Deportes,2305,2.0,"Primary: #374df5, Secondary: #374df5","Primary: #ffffcc, Secondary: #ffffcc",...,Jorge Fossati,1.0,0.0,0.0,2.0,1 - 2,Estadio Union Tarma,Tarma,2025-10-26 20:30:00,Kevin Ortega
3,14841185,https://www.sofascore.com/melgar-sport-huancay...,Melgar,2308,2.0,Sport Huancayo,33895,0.0,"Primary: #000000, Secondary: #ff0000","Primary: #ff0000, Secondary: #ffff00",...,Richard Pellejero,1.0,1.0,0.0,0.0,2 - 0,Estadio de la UNSA,Arequipa,2025-10-26 23:00:00,Víctor Cori
4,14841189,https://www.sofascore.com/alianza-lima-melgar/...,Alianza Lima,2311,2.0,Melgar,2308,2.0,"Primary: #091d47, Secondary: #ffffff","Primary: #000000, Secondary: #ff0000",...,Juan Maximo Reynoso,1.0,1.0,1.0,1.0,2 - 2,Estadio Alejandro Villanueva,Lima,2025-11-01 01:00:00,Pablo Lopez Ramos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,13352931,https://www.sofascore.com/alianza-lima-cusco-f...,Alianza Lima,2311,3.0,Cusco FC,63760,0.0,"Primary: #091d47, Secondary: #ffffff","Primary: #00ffff, Secondary: #33ffff",...,Miguel Rondelli,2.0,1.0,0.0,0.0,3 - 0,Estadio Alejandro Villanueva,Lima,2025-02-09 00:00:00,Daniel Ureta
334,13352938,https://www.sofascore.com/alianza-universidad-...,Alianza Universidad,306660,2.0,Club Sporting Cristal,2302,2.0,"Primary: #ff0000, Secondary: #0000c8","Primary: #33ccff, Secondary: #ffffff",...,Paulo Autuori,2.0,0.0,2.0,0.0,2 - 2,Estadio Heraclio Tapia,Huanuco,2025-02-09 18:00:00,Jordi Espinoza
335,13352936,https://www.sofascore.com/sport-boys-adc-juan-...,Sport Boys,2312,1.0,ADC Juan Pablo II,511206,0.0,"Primary: #ffc0db, Secondary: #000000","Primary: #374df5, Secondary: #374df5",...,Santiago Acasiete,0.0,1.0,0.0,0.0,1 - 0,Estadio Miguel Grau,Callao,2025-02-09 20:00:00,Roberto Perez Gutierrez
336,13352932,https://www.sofascore.com/comerciantes-unidos-...,Comerciantes Unidos,213609,1.0,Universitario de Deportes,2305,1.0,"Primary: #ffff00, Secondary: #ffff00","Primary: #ffffcc, Secondary: #ffffcc",...,Jorge Fossati,0.0,1.0,0.0,1.0,1 - 1,Estadio Juan Maldonado Gamarra,Distrito de Cutervo,2025-02-09 20:30:00,Micke Palomino


In [8]:
# Guardar Excel en la carpeta actual
nombre_archivo = f"Partidos_{liga}_{year}.xlsx"
df_matches.to_excel(nombre_archivo, index=False, engine='openpyxl')

# Generar enlace de descarga en Jupyter
from IPython.display import FileLink
FileLink(nombre_archivo)

c:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStatz\notebooks\Partidos_Liga 1 Peru_2025.xlsx